In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
sns.set_theme(style="darkgrid")

In [ ]:
exptime = "2024-09-17-23:08:02-45c5a874d54b0c128a3e4511da4ce5356b6688b4"
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
directory_path = os.path.join(parent_dir, "results", exptime)
files = os.listdir(directory_path)
file_names = [file for file in files if os.path.isfile(os.path.join(directory_path, file))]
file_names = [file_name for file_name in file_names if file_name != "rerun.txt"]
print(len(file_names))

In [ ]:
rows = []
dicts = []

#['20', 'client_open', 'ms', 'rule_mongodb', 'YCSB_A', '12c1s5r1p', 'rw_zipf_0.95', 'concurrent_100', '0', '1000000', '0', '1000', '25', '30.res']
#['20', 'client_open', 'ms', 'rule_fpga_raft', '12c1s5r1p', 'rw_zipf_1', 'concurrent_40', '0', '1000000', '0', '1000', '25', '30.res']
for file_name in file_names:
    d = {}
    parts = file_name.split("-")
    # print(parts)
    d["latency"] = latency = parts[0]
    d["mode"] = mode = parts[3]
    d["ycsb"] = ycsb = parts[4]
    d["site"] = site = parts[5]
    d["workload"] = workload = parts[6]
    d["conc"] = conc = int(parts[7][parts[7].find("_")+1:])
    d["finish_countdown"] = finish_countdown = parts[8]
    d["fastpath_timeout"] = fastpath_timeout = parts[9]
    d["wait_commit_timeout"] = wait_commit_timeout = parts[10]
    d["instance_commit_timeout"] = instance_commit_timeout = parts[11]
    d["fastpath_rate"] = fastpash_mode = "adaptive" if parts[10] == "101" else parts[12]
    d["duration"] = duration = parts[13][:parts[13].find(".")]
    
    if "plus" not in mode:
        finish_countdown = None
        fastpath_timeout = None
        wait_commit_timeout = None
        instance_commit_timeout = None

    d["throughput"] = throughtput = None
    d["mid_throughput"] = mid_throughtput = None
    d["original_throughput"] = original_throughput = None

    d["fastpath_count"] = fastpath_count = 0
    d["fastpath_50pct"] = fastpath_50pct = None
    d["fastpath_90pct"] = fastpath_90pct = None
    d["fastpath_99pct"] = fastpath_99pct = None
    d["fastpath_ave"] = fastpath_ave = None

    d["original_protocol_count"] = original_protocol_count = 0
    d["original_protocol_50pct"] = original_protocol_50pct = None
    d["original_protocol_90pct"] = original_protocol_90pct = None
    d["original_protocol_99pct"] = original_protocol_99pct = None
    d["original_protocol_ave"] = original_protocol_ave = None

    d["all_original_count"] = all_original_count = 0
    d["all_original_50pct"] = all_original_50pct = None
    d["all_original_90pct"] = all_original_90pct = None
    d["all_original_99pct"] = all_original_99pct = None
    d["all_original_ave"] = all_original_ave = None

    d["latency50pct"] = latency50pct = None
    d["latency90pct"] = latency90pct = None
    d["latency99pct"] = latency99pct = None
    d["latency_ave"] = latency_ave = None

    d["cpu5_medium"] = cpu5_medium = -1
    d["cpu6_medium"] = cpu6_medium = -1
    d["cpu7_medium"] = cpu7_medium = -1    
    d["cpu8_medium"] = cpu8_medium = -1
    d["cpu9_medium"] = cpu9_medium = -1
    d["clientall_medium"] = clientall_medium = -1

    d["fastpath_attempted"] = fastpath_attempted = 0
    d["fastpath_successed"] = fastpath_successed = 0

    d["memory_before"] = memory_before = 0
    d["memory_during"] = memory_during = 0

    d["witness0_50pct"] = d["witness0_90pct"] = d["witness0_99pct"] = d["witness0_ave"] = -1
    d["witness1_50pct"] = d["witness1_90pct"] = d["witness1_99pct"] = d["witness1_ave"] = -1
    d["witness2_50pct"] = d["witness2_90pct"] = d["witness2_99pct"] = d["witness2_ave"] = -1
    witness0_50pct = witness0_90pct = witness0_99pct = witness0_ave = -1
    witness1_50pct = witness1_90pct = witness1_99pct = witness1_ave = -1
    witness2_50pct = witness2_90pct = witness2_99pct = witness2_ave = -1
    
    with open(os.path.join(directory_path, file_name), "r") as file:
        for line in file:
            if "Total throughtput is" in line:
                d["throughput"] = throughtput = float(line[line.find("is")+3:].strip())
            if "Mid throughput is" in line:
                d["mid_throughput"] = mid_throughtput = float(line[line.find("is")+3:].strip())
            if "Original throughput is" in line:
                d["original_throughput"] = original_throughput = float(line[line.find("is")+3:].strip())
            if "Fastpath count" in line:
                d["fastpath_count"] = fastpath_count = int(line[line.find("count")+6:line.find("0pct")])
                d["fastpath_50pct"] = fastpath_50pct = float(line[line.find("50pct")+6:line.find("90pct")]) if fastpath_count > 0 else None
                d["fastpath_90pct"] = fastpath_90pct = float(line[line.find("90pct")+6:line.find("99pct")]) if fastpath_count > 0 else None
                d["fastpath_99pct"] = fastpath_99pct = float(line[line.find("99pct")+6:line.find("ave")]) if fastpath_count > 0 else None
                d["fastpath_ave"] = fastpath_ave = float(line[line.find("ave")+4:]) if fastpath_count > 0 else None
            if "Original-Protocol" in line:
                d["original_protocol_count"] = original_protocol_count = int(line[line.find("count")+6:line.find("0pct")])
                d["original_protocol_50pct"] = original_protocol_50pct = float(line[line.find("50pct")+6:line.find("90pct")]) if original_protocol_count > 0 else None
                d["original_protocol_90pct"] = original_protocol_90pct = float(line[line.find("90pct")+6:line.find("99pct")]) if original_protocol_count > 0 else None
                d["original_protocol_99pct"] = original_protocol_99pct = float(line[line.find("99pct")+6:line.find("ave")]) if original_protocol_count > 0 else None
                d["original_protocol_ave"] = original_protocol_ave = float(line[line.find("ave")+4:]) if original_protocol_count > 0 else None
            if "All original" in line:
                d["all_original_count"] = all_original_count = int(line[line.find("count")+6:line.find("0pct")])
                d["all_original_50pct"] = all_original_50pct = float(line[line.find("50pct")+6:line.find("90pct")]) if all_original_count > 0 else None
                d["all_original_90pct"] = all_original_90pct = float(line[line.find("90pct")+6:line.find("99pct")]) if all_original_count > 0 else None
                d["all_original_99pct"] = all_original_99pct = float(line[line.find("99pct")+6:line.find("ave")]) if all_original_count > 0 else None
                d["all_original_ave"] = all_original_ave = float(line[line.find("ave")+4:]) if all_original_count > 0 else None
            if "Latency-50pct is" in line:
                d["latency50pct"] = latency50pct = float(line[line.find("Latency-50pct is")+17:line.find("Latency-90pct is")-4])
                d["latency90pct"] = latency90pct = float(line[line.find("Latency-90pct is")+17:line.find("Latency-99pct is")-4])
                d["latency99pct"] = latency99pct = float(line[line.find("Latency-99pct is")+17:line.find("ave is")-4])
                d["latency_ave"] = latency_ave = float(line[line.find("ave is")+7:-3])
            if "Fastpath statistics" in line:
                d["fastpath_attempted"] = fastpath_attempted = int(line[line.find("attempted")+10:line.find("successed")])
                d["fastpath_successed"] = fastpath_successed = int(line[line.find("successed")+10:line.find("rate(pct)")])
            if "cpu5 " in line:
                d["cpu5_medium"] = cpu5_medium = float(line[line.find("medium")+8:line.find("mean")])
            if "cpu6 " in line:
                d["cpu6_medium"] = cpu6_medium = float(line[line.find("medium")+8:line.find("mean")])
            if "cpu7 " in line:
                d["cpu7_medium"] = cpu7_medium = float(line[line.find("medium")+8:line.find("mean")])
            if "cpu8 " in line:
                d["cpu8_medium"] = cpu8_medium = float(line[line.find("medium")+8:line.find("mean")])
            if "cpu9 " in line:
                d["cpu9_medium"] = cpu9_medium = float(line[line.find("medium")+8:line.find("mean")])
            if "Client sum" in line:
                d["clientall_medium"] = clientall_medium = float(line[line.find("medium")+8:line.find("mean")])
            if "witness size" in line:
                if "loc_id=0" in line:
                    d["witness0_50pct"] = witness0_50pct = float(line[line.find("50pct")+6:line.find("90pct")])
                    d["witness0_90pct"] = witness0_90pct = float(line[line.find("90pct")+6:line.find("99pct")])
                    d["witness0_99pct"] = witness0_99pct = float(line[line.find("99pct")+6:line.find("ave")])
                    d["witness0_ave"]   = witness0_ave   = float(line[line.find("ave")+4:])
                if "loc_id=1" in line:
                    d["witness1_50pct"] = witness1_50pct = float(line[line.find("50pct")+6:line.find("90pct")])
                    d["witness1_90pct"] = witness1_90pct = float(line[line.find("90pct")+6:line.find("99pct")])
                    d["witness1_99pct"] = witness1_99pct = float(line[line.find("99pct")+6:line.find("ave")])
                    d["witness1_ave"]   = witness1_ave   = float(line[line.find("ave")+4:])
                if "loc_id=2" in line:
                    d["witness2_50pct"] = witness2_50pct = float(line[line.find("50pct")+6:line.find("90pct")])
                    d["witness2_90pct"] = witness2_90pct = float(line[line.find("90pct")+6:line.find("99pct")])
                    d["witness2_99pct"] = witness2_99pct = float(line[line.find("99pct")+6:line.find("ave")])
                    d["witness2_ave"]   = witness2_ave   = float(line[line.find("ave")+4:])
            if "memory before test" in line:
                d["memory_before"] = float(line[line.find("test:")+6:])
            if "memory during test" in line:
                d["memory_during"] = float(line[line.find("test:")+6:])
                
                
    # print(mode, site, workload, conc, throughtput, latency50pct, latency90pct, latency99pct, fastpath_count, coordinatoraccept_count, original_count, max_gap)
    
    d["total_count"] = total_count = fastpath_count + original_protocol_count
    d["fastpath_success_pct"] = fastpath_success_pct = d["fastpath_successed"] * 100.0 / d["fastpath_attempted"] if d["fastpath_attempted"] > 0 else 0
    d["cpu_max"] = max(d["cpu5_medium"], d["cpu6_medium"], d["cpu7_medium"], d["cpu8_medium"], d["cpu9_medium"])
    d["cpu_ave"] = (d["cpu5_medium"] + d["cpu6_medium"] + d["cpu7_medium"] + d["cpu8_medium"] + d["cpu9_medium"]) / 3
    
    rows.append([latency, site, ycsb, mode, workload, fastpash_mode, duration, conc, finish_countdown, fastpath_timeout, wait_commit_timeout, \
                 instance_commit_timeout, throughtput, mid_throughtput, original_throughput, fastpath_count, fastpath_50pct, fastpath_90pct, fastpath_99pct, fastpath_ave, fastpath_success_pct, \
                 original_protocol_count, original_protocol_50pct, original_protocol_90pct, original_protocol_99pct, original_protocol_ave, \
                 all_original_count, all_original_50pct, all_original_90pct, all_original_99pct, all_original_ave, total_count, \
                 latency50pct, latency90pct, latency99pct, latency_ave, cpu5_medium, cpu6_medium, cpu7_medium, cpu8_medium, cpu9_medium, clientall_medium, \
                 witness0_50pct, witness0_90pct, witness0_99pct, witness0_ave, witness1_50pct, witness1_90pct, witness1_99pct, witness1_ave, \
                 witness2_50pct, witness2_90pct, witness2_99pct, witness2_ave])

    # if duration != "120":
    #     continue
    
    dicts.append(d)

print(len(rows))
print(len(dicts))

In [ ]:
fields = ["latency", "site",'ycsb', "mode", "workload", "fastpath_mode", "duration", "conc", "finish_countdown", "fastpath_timeout", "wait_commit_timeout", \
          "instance_commit_timeout", "throughtput", "mid_throughput", "original_throughput", "fastpath_count", "fastpath_50pct", "fastpath_90pct", "fastpath_99pct", "fastpath_ave", "fastpath_success_pct", \
          "original_protocol_count", "original_protocol_50pct", "original_protocol_90pct", "original_protocol_99pct", "original_protocol_ave", \
          "all_original_count", "all_original_50pct", "all_original_90pct", "all_original_99pct", "all_original_ave", "total_count", \
          "latency50pct", "latency90pct", "latency99pct", "latency_ave", "cpu5_medium", "cpu6_medium", "cpu7_medium",  "cpu8_medium", "cpu9_medium", "clientall_medium", \
          "witness0_50pct", "witness0_90pct", "witness0_99pct", "witness0_ave", "witness1_50pct", "witness1_90pct", "witness1_99pct", "witness1_ave", \
          "witness2_50pct", "witness2_90pct", "witness2_99pct", "witness2_ave"]

import csv 

filename = os.path.join("curp_results-" + exptime + ".csv")

# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [ ]:
pdf_files = glob.glob("*.pdf")

for pdf_file in pdf_files:
    os.remove(pdf_file)
    print(f"Removed: {pdf_file}")

In [ ]:
# sites = ["12c1s5r1p", "12c1s3r1p"]
sites = ["12c1s5r1p"]
# workloads = ["rw_1000000", "rw_zipf_0", "rw_zipf_0.2", "rw_zipf_0.4", "rw_zipf_0.6", "rw_zipf_0.8", "rw_zipf_1"]
# workloads = ["rw_1", "rw_1000"]
workloads = ["rw_zipf_0.5", "rw_zipf_0.55", "rw_zipf_0.6", "rw_zipf_0.65", "rw_zipf_0.7", "rw_zipf_0.75", "rw_zipf_0.8", "rw_zipf_0.85", "rw_zipf_0.9", "rw_zipf_0.95",  "rw_zipf_1"]
# workloads = ["rw_1000000", "rw_zipf_1"]
pcts = ["0", "25", "50", "75", "100"]
ycsb_set = ["YCSB_A", "YCSB_B"]
# pcts = ["80", "90", "99", "99.9", "100"]
print(pcts)




def get_data_by_mode(pattern):
    data = {}
    data["vanilla"] = [e for e in dicts if pattern in e["mode"] and "none" in e["mode"]]
    for pct in pcts:
        data[pct] = [e for e in dicts if "rule" in e["mode"] and pattern in e["mode"] and e["fastpath_rate"] == pct]
    return data
    
raft_data = get_data_by_mode("raft")
copilot_data = get_data_by_mode("copilot")
mencius_data = get_data_by_mode("mencius")
mongodb_data = get_data_by_mode("mongodb")




print_for_paper = False

latex_constants = {}

In [ ]:
raft_data["25"]

In [ ]:
pct_line_infos = [
    ["vanilla", '#437c17', "D"],
    ["0%", "black", "x"], ##f5f5dc
    ["25%", "#8b7e66", "^"],
    ["50%", "#245878", "s"],
    ["75%", "#5e5749", "P"],
    ["100%", "#a8001f", "o"],
    # ["adaptive", "black", "x"],
]
# pct_line_infos = [
#     ["vanilla", '#437c17', "D"],
#     ["80%", "black", "x"], ##f5f5dc
#     ["90%", "#8b7e66", "^"],
#     ["99%", "#245878", "s"],
#     ["99.9%", "#5e5749", "P"],
#     ["100%", "#a8001f", "o"],
#     # ["adaptive", "black", "x"],
# ]

line_width = 0.7

legend_draw = False

# filter a specific conc number when x-axis is not number of clients
fixed_conc_number = 10

In [ ]:
x_ticks = [_.split('_')[-1] for _ in workloads]

workload_to_index = {}
for i, workload in enumerate(workloads):
    workload_to_index[workload] = i

y_limit = [(0,100), (0,120), (0, 200), (0,100)]

def draw_latency_ycsb(axes, ycsb, data, title, y_label, limit, x_label):
    l = {}
    for pct_index, pct in enumerate(["vanilla"] + pcts):
        l[pct_index] = {}
        for e in data[pct]:
            if e["ycsb"] == ycsb:
               workload_index = workload_to_index[e["workload"]]
               l[pct_index][workload_index] = e["latency_ave"]
        this_line = [l[pct_index][i] for i in range(len(workloads))]
        axes.plot(x_ticks, this_line, label=pct_line_infos[pct_index][0], color=pct_line_infos[pct_index][1], marker=pct_line_infos[pct_index][2], lw=line_width)
    
    axes.legend(bbox_to_anchor = (1.2, 0.2), loc='center right',fontsize='x-small')
    axes.set_title(title + " " + ycsb)
    axes.set_ylabel(y_label)
    axes.set_xlabel(x_label)
    # axes.set_ylim(limit)
protocols = [
    (raft_data, "Raft", y_limit[0]),
    (copilot_data, "Copilot", y_limit[1]),
    (mencius_data, "Mencius", y_limit[2]),
    (mongodb_data, "MongodB", y_limit[3])
]
for ycsb in ycsb_set:

    fig_latency, axes_latency = plt.subplots(nrows=1, ncols=4, figsize=(28, 4))
    for i, (data, title, limit) in enumerate(protocols):
        draw_latency_ycsb(axes_latency[i], ycsb, data, title, "Latencies(ms)", limit,"Zipfian Skew Parameter($\\theta$)")

    fig_latency.subplots_adjust(left=0,wspace=0.4)
    # fig_latency.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15))
    fig_latency.savefig("latency_pct_" + site + "_" + ycsb + ".pdf", bbox_inches='tight')
    
    fig_latency.show()


In [ ]:
x_ticks = [_.split('_')[-1] for _ in workloads]

workload_to_index = {}
for i, workload in enumerate(workloads):
    workload_to_index[workload] = i

y_limit = [(0,100), (0,120), (0, 200), (0,100)]

def draw_latency_ycsb(axes, ycsb, data, title, y_label, limit, x_label):
    l = {}
    for pct_index, pct in enumerate(["vanilla"] + pcts):
        l[pct_index] = {}
        for e in data[pct]:
            if e["ycsb"] == ycsb:
               workload_index = workload_to_index[e["workload"]]
               l[pct_index][workload_index] = e["fastpath_success_pct"]
        this_line = [l[pct_index][i] for i in range(len(workloads))]
        axes.plot(x_ticks, this_line, label=pct_line_infos[pct_index][0], color=pct_line_infos[pct_index][1], marker=pct_line_infos[pct_index][2], lw=line_width)
    
    axes.legend(loc='center left',fontsize='x-small')
    axes.set_title(title + " " + ycsb)
    axes.set_ylabel(y_label)
    axes.set_xlabel(x_label)
    # axes.set_ylim(limit)
protocols = [
    (raft_data, "Raft", y_limit[0]),
    (copilot_data, "Copilot", y_limit[1]),
    (mencius_data, "Mencius", y_limit[2]),
    (mongodb_data, "MongodB", y_limit[3])
]
for ycsb in ycsb_set:

    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(24, 4))
    for i, (data, title, limit) in enumerate(protocols):
        draw_latency_ycsb(axes[i], ycsb, data, title, "Latencies(ms)", limit, "Zipfian Skew Parameter($\\theta$)")

    fig.subplots_adjust(left=0)
    fig.savefig("success_rate_" + site + "_" + ycsb + ".pdf", bbox_inches="tight")
    fig.show()
